# Data modeling y evaluation (II)
A lo largo de este notebook se cubre parte de la cuarta y quinta fase de la metodología CRIPS-DM, en este caso el modelado y la evaluación de los resultados


1. **Comprensión del Negocio (Business Understanding)**
   - Consistente en el entendimiento del objetivo del proyecto.

2. **Comprensión de los Datos (Data Understanding)**
   - Relacionada con la carga y primera evaluación del conjunto de datos. Se divide a su vez en :


3. **Preparación de los Datos (Data Preparation)** 
   - Consistente en la limpieza, preparación y extracción de características de los datos.

4. <span style="color:#66FF99;">**Modelado (Modeling)**  </span> 


   Relacionada con la selección del modelo y el ajuste hiperparamétrico del mismo. En este caso, se han probado dos modelos diferentes, donde en este notebook se desarrolla la implementación del primero:


   4.1. Primer modelo baseline inicial: se hace uso de la arquitectura U-Net entrenada con el dataset en bruto.


   <span style="color:#66FF99;">**4.2. Modelo final: ensemble de modelos, que combina las salidas de Fast-SAM y CLIP con la arquitectura U-Net para resolver el problema de segmentación.**</span> 

5. <span style="color:#66FF99;">**Evaluación (Evaluation)**</span>  
   - Evaluación de los resultados obtenidos por el modelo.

6. **Implementación (Deployment)**  
   - Integración del modelo de forma que sea accesible para su uso.

### Data modeling

En este apartado se realiza una el entrenamiento de un primer modelo que nos servirá como baseline, que en este caso se trata de el uso de la arquitectura U-Net, adaptada al número de canales de salida para la obtención de las máscaras con cada una de las clases de salida correspondiente.


En primera instancia es necesario la generación eficiende del pipeline que carga las imágenes de COCO y sus correspondientes máscaras en memoria. 


Para ello, se cargan las librerias a usar a lo largo del notebook

In [ ]:
import tensorflow as tf
import numpy as np
from pycocotools.coco import COCO
import os
import cv2

In [ ]:
IMAGE_SIZE = (128, 128)  # Resize images to this size
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
OUTPUT_MODEL_FOLDER = "my_trained_models" # Nombre del directorio donde se almacenan los pesos de los modeos a guardar

In [ ]:
def load_and_preprocess_image(image_id):
    image_info = coco.loadImgs(image_id)[0]
    image_path = os.path.join(coco_images_dir, image_info['file_name'])
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = image / 255.0  # Normalize to [0, 1]
    return image

In [ ]:
def load_and_preprocess_mask(image_id):
    mask_path = os.path.join(coco_masks_dir, f"{image_id}.png")
    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=1)  # Single channel mask
    mask = tf.image.resize(mask, IMAGE_SIZE, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    mask = tf.cast(mask, tf.int32)
    mask = tf.one_hot(mask[..., 0], NUM_CLASSES)  # One-hot encode the mask
    return mask

In [ ]:
def coco_generator():
    for image_id in image_ids:
        yield load_and_preprocess_image(image_id), load_and_preprocess_mask(image_id)

In [ ]:
dataset = tf.data.Dataset.from_generator(
    coco_generator,
    output_signature=(
        tf.TensorSpec(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3), dtype=tf.float32),
        tf.TensorSpec(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], NUM_CLASSES), dtype=tf.float32)
    )
)

In [ ]:
dataset = dataset.cache() \
                 .shuffle(buffer_size=1000) \
                 .batch(BATCH_SIZE) \
                 .prefetch(buffer_size=AUTOTUNE)

In [ ]:
def build_unet(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    # Bottleneck
    b1 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    b1 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(b1)

    # Decoder
    u1 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(b1)
    u1 = tf.keras.layers.concatenate([u1, c2])
    c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c3)

    u2 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c3)
    u2 = tf.keras.layers.concatenate([u2, c1])
    c4 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    c4 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c4)

    outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(c4)

    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
def dice_coefficient(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1, 2))
    union = tf.reduce_sum(y_true + y_pred, axis=(1, 2))
    dice = (2. * intersection + 1e-7) / (union + 1e-7)
    return tf.reduce_mean(dice)

In [ ]:
def iou_metric(y_true, y_pred):
    intersection = tf.reduce_sum(y_true * y_pred, axis=(1, 2))
    union = tf.reduce_sum(y_true + y_pred, axis=(1, 2)) - intersection
    iou = (intersection + 1e-7) / (union + 1e-7)
    return tf.reduce_mean(iou)

In [ ]:
input_shape = (IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
model = build_unet(input_shape, NUM_CLASSES)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy', dice_coefficient, iou_metric])

In [ ]:
model.fit(dataset, epochs=10)


In [ ]:
evaluation = model.evaluate(dataset)
print(f"Loss: {evaluation[0]}, Accuracy: {evaluation[1]}, Dice Coefficient: {evaluation[2]}, IoU: {evaluation[3]}")